In [68]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # pytorch has a lot of datasets # https://docs.pytorch.org/vision/stable/datasets.html
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter # to print to tensorboard
import torchvision

In [69]:
# Create Simple Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # (28*28 = 784)
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [70]:
# TODO: Create simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))    
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [71]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [72]:
# Hyperparameters
in_channel = 1
num_classes = 10
# learning_rate = 0.001
# batch_size = 64
num_epochs = 1 # 5

In [73]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [74]:
# Initialize network
    # model = CNN().to(device)

In [75]:
# loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [76]:
# writer = SummaryWriter(f'runs/MNIST/tryingout_tensorboard') 

# terminal: tensorboard --logdir runs

In [77]:
# batch_sizes = [2, 64, 1024]
# learning_rates = [0.1, 0.01, 0.001, 0.0001]

# batch_sizes = [256] # [64]
# learning_rates = [0.0001]
# classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# for batch_size in batch_sizes:
#     for learning_rate in learning_rates:
#         step = 0
#         # Initialize network
#         model = CNN(in_channels=in_channel, num_classes=num_classes)
#         model.to(device)
#         model.train()
#         writer = SummaryWriter(f'runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}') 
#         criterion = nn.CrossEntropyLoss()
#         optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#         train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#         for epoch in range(num_epochs):
#             losses = []
#             accuracies = []
            
#             for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
#                 # Get data to cuda if possible
#                 data = data.to(device=device)
#                 targets = targets.to(device=device)
        
#                 # forward
#                 scores = model(data)
#                 loss = criterion(scores, targets)
#                 losses.append(loss.item())
        
#                 # backward
#                 optimizer.zero_grad()
#                 loss.backward()
#                 # gradient descent or adam step
#                 optimizer.step()

#                 # Calculate 'running' training accuracy
#                 features = data.reshape(data.shape[0], -1)
#                 img_grid = torchvision.utils.make_grid(data)
#                 _, predictions = scores.max(1)
#                 num_correct = (predictions == targets).sum()
#                 running_training_acc = float(num_correct)/float(data.shape[0])
#                 accuracies.append(running_training_acc)

#                 # Plot things to tensorboard
#                 class_labels = [classes[label] for label in predictions]
#                 writer.add_image('mnist_images', img_grid)
#                 writer.add_histogram('fc1', model.fc1.weight)
#                 writer.add_scalar('Training loss', loss, global_step=step)
#                 writer.add_scalar('Training Accuracy', running_training_acc, global_step=step)

#                 if batch_idx == 230:
#                     writer.add_embedding(features, metadata=class_labels, 
#                                          label_img = data, global_step=batch_idx)
#                 step += 1

#             final_accuracy = sum(accuracies)/len(accuracies)
#             final_loss = sum(losses)/len(losses)

#             writer.add_hparams({ 'lr' : learning_rate, 'bsize': batch_size },
#                                    { 'accuracy': final_accuracy, 
#                                    'loss': final_loss })
#             print(f'Mean loss of this epoch was {sum(losses)/len(losses)}')

100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [00:33<00:00,  7.03it/s]

Mean loss of this epoch was 2.0079509258270263


100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [00:35<00:00,  6.67it/s]

Epoch 0: Mean loss = 2.0589134180799444


In [50]:
# Train network

# step = 0
# for epoch in range(num_epochs):
#     losses = []
#     accuracies = []
    
#     for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
#         # Get data to cuda if possible
#         data = data.to(device=device)
#         targets = targets.to(device=device)

#         # forward
#         scores = model(data)
#         loss = criterion(scores, targets)
#         losses.append(loss.item())

#         # backward
#         optimizer.zero_grad()
#         loss.backward()
#         # gradient descent or adam step
#         optimizer.step()

#         _, predictions = scores.max(1)
#         num_correct = (predictions == targets).sum()
#         running_training_acc = float(num_correct)/float(data.shape[0])

#         writer.add_scalar('Training loss', loss, global_step=step)
#         writer.add_scalar('Training Accuracy', running_training_acc, global_step=step)
#         step += 1

#     print(f'Mean loss of this epoch was {sum(losses)/len(losses)}')

In [82]:
batch_sizes = [256]
learning_rates = [0.0001]
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        model = CNN(in_channels=in_channel, num_classes=num_classes)
        model.to(device)
        model.train()
        writer = SummaryWriter(f'runs/MNIST/MiniBatchSize {batch_size} LR {learning_rate}') 
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            
            for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
                data = data.to(device=device)
                targets = targets.to(device=device)
        
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())
        
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_training_acc = float(num_correct)/float(data.shape[0])
                accuracies.append(running_training_acc)
                
                # Log scalars and images
                if batch_idx % 100 == 0:  # Log less frequently
                    img_grid = torchvision.utils.make_grid(data)
                    writer.add_image('mnist_images', img_grid, global_step=step)
                    writer.add_histogram('fc1', model.fc1.weight, global_step=step)
                
                writer.add_scalar('Training loss', loss, global_step=step)
                writer.add_scalar('Training Accuracy', running_training_acc, global_step=step)
                step += 1
            
            # Add embedding ONCE per epoch (or at the end of training)
            # Get a sample batch for embedding visualization
            sample_data, sample_targets = next(iter(train_loader))
            sample_data = sample_data.to(device)
            features = sample_data.reshape(sample_data.shape[0], -1)
            class_labels = [classes[int(label)] for label in sample_targets]
            
            writer.add_embedding(
                features,
                metadata=class_labels,
                label_img=sample_data,
                global_step=epoch,
                tag='mnist_embedding'
            )
            
            final_accuracy = sum(accuracies)/len(accuracies)
            final_loss = sum(losses)/len(losses)
            writer.add_hparams(
                {'lr': learning_rate, 'bsize': batch_size},
                {'accuracy': final_accuracy, 'loss': final_loss}
            )
            print(f'Mean loss of this epoch was {final_loss}')
        
        writer.close()  # Important: close the writer

100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [00:16<00:00, 14.41it/s]


Mean loss of this epoch was 2.0136260889946147


In [14]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            scores.max()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59220 / 60000 with accuracy 98.70
Checking accuracy on test data
Got 9843 / 10000 with accuracy 98.43
